In [47]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [48]:
df = pd.read_csv('diabetes.csv')

In [49]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [50]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [51]:
x = df.drop(columns='Outcome')
y = df['Outcome']

In [52]:
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()

x = scale.fit_transform(x)

In [53]:
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=.2,random_state=42)

In [54]:
feature_cols = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin',
                'BMI','DiabetesPedigreeFunction','Age']

x_train = pd.DataFrame(x_train, columns=feature_cols)
x_test = pd.DataFrame(x_test, columns=feature_cols)

from sklearn.impute import SimpleImputer
cols_with_zero = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']

for col in cols_with_zero:
    x_train[col] = x_train[col].replace(0, np.nan)
    x_test[col] = x_test[col].replace(0, np.nan)



imp = SimpleImputer(strategy='median')
x_train = pd.DataFrame(imp.fit_transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(imp.transform(x_test), columns=x_test.columns)


In [55]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

m = LogisticRegression()

m.fit(x_train,y_train)

yy = m.predict(x_test)

accuracy_score(y_test,yy)

0.7532467532467533

In [56]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score, confusion_matrix

rf = RandomForestClassifier(random_state=42)

param = {
    'n_estimators' : np.arange(20,300,30),
    'max_depth' : np.arange(3,40,30),
    'criterion' : ['gini','entropy']
}

search  = RandomizedSearchCV(
    rf,
    param,
    scoring ='f1',
    n_iter=40,
    n_jobs=1,
    random_state=42
)

In [57]:
search.fit(x_train,y_train)

,estimator,RandomForestC...ndom_state=42)
,param_distributions,"{'criterion': ['gini', 'entropy'], 'max_depth': array([ 3, 33]), 'n_estimators': array([ 20, ...30, 260, 290])}"
,n_iter,40
,scoring,'f1'
,n_jobs,1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [58]:
search.best_score_

np.float64(0.6617674035606524)

In [59]:
model = search.best_estimator_

In [60]:
y_pred = model.predict(x_test)

In [61]:
accuracy_score(y_test,y_pred)

0.7402597402597403

In [62]:
from sklearn.svm import SVC

svm = SVC(kernel='linear')

svm.fit(x_train,y_train)

y_pre = svm.predict(x_test)

accuracy_score(y_test,y_pre)

0.7597402597402597

In [63]:
from sklearn.ensemble import AdaBoostClassifier

ada  = AdaBoostClassifier()

ada.fit(x_train,y_train)

y_pree = ada.predict(x_test)

accuracy_score(y_test,y_pree)


0.7792207792207793

In [65]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV

ada  = AdaBoostClassifier()

parameters = {
    'n_estimators': np.arange(75,200,15),
    'learning_rate': np.arange(.1,2,.3)
}

grid = GridSearchCV(
    ada,
    parameters,
    n_jobs=1,
    scoring='accuracy',
    cv=5
)

grid.fit(x_train,y_train)

,estimator,AdaBoostClassifier()
,param_grid,"{'learning_rate': array([0.1, 0....3, 1.6, 1.9]), 'n_estimators': array([ 75, ...65, 180, 195])}"
,scoring,'accuracy'
,n_jobs,1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,estimator,None


In [66]:
display(f'Best Training Score: {grid.best_score_}')
b_model = grid.best_estimator_
yb = b_model.predict(x_test)
display(f'Test Score: {accuracy_score(y_test,yb)}')

'Best Training Score: 0.7833933093429295'

'Test Score: 0.7597402597402597'

In [67]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.5, 1.0, 1.5],
}

grid = GridSearchCV(
    AdaBoostClassifier(),
    params,
    scoring='accuracy',
    cv=5,
    n_jobs=-1
)

grid.fit(x_train, y_train)

print(grid.best_params_)
print(grid.best_score_)


{'learning_rate': 1.0, 'n_estimators': 100}
0.7817939490870318
